In [1]:
import numpy as np
import scipy.io

In [ ]:
# Gaussian Random Field (GRF) function
def GRF(N, m, gamma, tau, sigma, type):
    if type == "dirichlet":
        m = 0

    my_const = 2 * np.pi if type == "periodic" else np.pi
    my_eigs = np.sqrt(2) * (np.abs(sigma) * ((my_const * np.arange(1, N + 1))**2 + tau**2)**(-gamma / 2))

    if type == "dirichlet":
        alpha = np.zeros(N)
    else:
        xi_alpha = np.random.randn(N)
        alpha = my_eigs * xi_alpha

    if type == "neumann":
        beta = np.zeros(N)
    else:
        xi_beta = np.random.randn(N)
        beta = my_eigs * xi_beta

    a = alpha / 2
    b = -beta / 2

    c = np.concatenate([np.flipud(a) - np.flipud(b) * 1j, [m + 0j], a + b * 1j])

    # Simplified chebfun-like function for demonstration
    def chebfun(x, c):
        return np.polynomial.chebyshev.chebval(x, c)

    if type == "periodic":
        uu = chebfun(np.linspace(0, 1, N), c)
        u = lambda t: uu(t - 0.5)
    else:
        uu = chebfun(np.linspace(-np.pi, np.pi, N), c)
        u = lambda t: uu(np.pi * t)

    return u

In [ ]:
# Burgers' equation solver (simplified version)
def burgers(init, tspan, s, visc):
    def lin(t, u):
        return visc * np.gradient(np.gradient(u))

    def nonlin(t, u):
        return -0.5 * np.gradient(u**2)

    def rhs(t, u):
        return lin(t, u) + nonlin(t, u)

    t_eval = np.linspace(tspan[0], tspan[1], s)
    sol = solve_ivp(rhs, tspan, init, t_eval=t_eval, method='RK45')
    return sol.y

In [ ]:
# Parameters
N = 10
gamma = 4
tau = 5
sigma = 25**2
visc = 0.01
s = 4096
steps = 100
nn = 101

input_data = np.zeros((N, nn))
if steps == 1:
    output_data = np.zeros((N, s))
else:
    output_data = np.zeros((N, steps, nn))

tspan = np.linspace(0, 1, steps + 1)
x = np.linspace(0, 1, s + 1)
X = np.linspace(0, 1, nn)

for j in range(N):
    u0 = GRF(s // 2, 0, gamma, tau, sigma, "periodic")
    u = burgers(u0(X), tspan, s, visc)

    u0_eval = u0(X)
    input_data[j, :] = u0_eval

    if steps == 1:
        output_data[j, :] = u
    else:
        for k in range(steps + 1):
            output_data[j, k, :] = u[k](X)

    print(j)

# Save results to a .mat file
scipy.io.savemat('Burger.mat', {'input': input_data, 'output': output_data, 'tspan': tspan, 'gamma': gamma, 'tau': tau, 'sigma': sigma})
